In [1]:
# First cell - Setup paths
import sys
import os

# Setup paths as before
notebook_dir = os.path.dirname('__file__')
lerobot_root = os.path.abspath(os.path.join(notebook_dir, '../..'))
sys.path.append(lerobot_root)
os.chdir(lerobot_root)

In [2]:
import cv2
import numpy as np
from PIL import Image
import torch
from tqdm.notebook import tqdm
from typing import List, Optional, Dict, Union, Tuple
import matplotlib.pyplot as plt
from pathlib import Path
import modal
import gc
import json
from collections import deque
import time
import re
from google import genai
from google.genai import types
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
from lerobot.caferacer.scripts.gemini_utils import *

objc[50084]: Class AVFFrameReceiver is implemented in both /Users/shreyas/opt/anaconda3/envs/caferacer/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x310eec798) and /Users/shreyas/opt/anaconda3/envs/caferacer/lib/libavdevice.59.7.100.dylib (0x325ea4778). One of the two will be used. Which one is undefined.
objc[50084]: Class AVFAudioReceiver is implemented in both /Users/shreyas/opt/anaconda3/envs/caferacer/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x310eec7e8) and /Users/shreyas/opt/anaconda3/envs/caferacer/lib/libavdevice.59.7.100.dylib (0x325ea47c8). One of the two will be used. Which one is undefined.


In [3]:
train_repo = "shreyasgite/so100_base_left"
eval_repo = "shreyasgite/eval_test"

In [4]:
gsam = modal.Function.lookup("grounded-sam","GroundedSam.run", environment_name='prod')

/Users/shreyas/opt/anaconda3/envs/caferacer/lib/python3.10/asyncio/events.py:80: DeprecationError: 2025-01-27: `modal.Function.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Function.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)
/Users/shreyas/opt/anaconda3/envs/caferacer/lib/python3.10/asyncio/events.py:80: DeprecationError: 2025-02-11: Looking up class methods using Function.from_name will be deprecated in a future version of Modal.
Use modal.Cls.from_name instead, e.g.

GroundedSam = modal.Cls.from_name("grounded-sam", "GroundedSam")
instance = GroundedSam(...)
instance.run.remote(...)

  self._context.run(self._callback, *self._args)


In [5]:
train_results = analyze_training_data(train_repo, episodes=list(range(8)))
train_summary = summarize_training_data(train_results)

Analyzing Training Episodes:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
print(train_summary)

Here's an analysis of the provided dataset, focusing on object distributions, success patterns, biases, and limitations, along with suggestions for data augmentation:

**1. Training Statistics:**

*   **Container Position:** In all 8 episodes (100%), the container is described as being on the *right* side of the image, relative to the robot arm, which is consistently in the center.  There are *no* instances of the container being on the left.
*   **Lego Brick Position:** In all 8 episodes (100%), the Lego bricks are on the *left* side of the image, relative to the robot arm. There are *no* instances of the bricks being on the right.
*   **Robot Arm Position:** The robot arm is consistently described as being in the "center" of the image in all episodes.
*   **Lego Brick Colors:** The Lego bricks are consistently described as green, yellow, or dark green.  There's no mention of other Lego colors.
*   **Container Color:** The container is always described as "blue." There are no other co

In [9]:
eval_results = analyze_eval_data(eval_repo, episodes=list(range(1,10)))

Analyzing Eval Episodes:   0%|          | 0/9 [00:00<?, ?it/s]

In [10]:
augmentations_raw = get_augmentations(eval_results, train_summary)
augmentations = parse_json(augmentations_raw)

In [17]:
augmentations

{'recommended_augmentations': {'flip_frame': True,
  'change_color': {'object': 'blue container', 'target_color': 'yellow'},
  'inpaint_distraction': {'objects': 'rubik cube'}},
 'reasoning': "The evaluation episodes, combined with the training data analysis, reveal several critical biases that need addressing.  \n\n1. **Positional Bias:** The training data consistently placed the container on the right and bricks on the left. While some evaluation episodes had the container on the left (Episode 2) or the brick on the right (Episode 2), the robot consistently failed when the configuration was different from the *dominant* training setup. This strongly suggests the robot overfit to the training data's positional arrangement.  `flip_frame` is essential to correct this.\n\n2. **Color Bias:** The training data exclusively used a blue container.  The evaluation introduced yellow containers (Episodes 3, 4, 5), and the robot failed in most cases where the container wasn't blue. This indicates

In [19]:
print(f"Reasoning: {augmentations['reasoning']}")
print(f"\n\nExpected_improvements: {augmentations['expected_improvements']}")

Reasoning: The evaluation episodes, combined with the training data analysis, reveal several critical biases that need addressing.  

1. **Positional Bias:** The training data consistently placed the container on the right and bricks on the left. While some evaluation episodes had the container on the left (Episode 2) or the brick on the right (Episode 2), the robot consistently failed when the configuration was different from the *dominant* training setup. This strongly suggests the robot overfit to the training data's positional arrangement.  `flip_frame` is essential to correct this.

2. **Color Bias:** The training data exclusively used a blue container.  The evaluation introduced yellow containers (Episodes 3, 4, 5), and the robot failed in most cases where the container wasn't blue. This indicates a strong color bias.  `change_color` specifically targeting the container is necessary.

3. **Distraction Sensitivity:** The presence of various background objects ('lamp', 'chair', 'wi

In [16]:
DATA_AUG = augmentations['recommended_augmentations']
DATA_AUG['inpaint_distraction']['objects'] = DATA_AUG['inpaint_distraction']['objects'][0]
print(DATA_AUG)

{'flip_frame': True, 'change_color': {'object': 'blue container', 'target_color': 'yellow'}, 'inpaint_distraction': {'objects': 'rubik cube'}}


In [18]:
#dataset0 = LeRobotDataset(train_repo)

In [ ]:
#dataset = create_dataset(train_repo, dataset0, gsam, DATA_AUG=DATA_AUG)
#dataset.push_to_hub()